# Scope

Use regression discontinuity design (RDD) to determine if there is a significant change in my average running pace since March 20, 2022. The data will be imported from my Garmin account, which I use to track my runs. The results may help me determine the reason for my chronic fatigue since that time.

In [1]:
# Import data

activity_data <- read.csv('Activities.csv')